### 1. Первичная выгрузка и обработка данных

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('reestr_new_TOTAL.csv').copy()
df

In [ ]:
import torch

# Проверяем доступность GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Используем: {device}')

In [ ]:
import numpy as np


''' Удаление столбца "ЖК", так как это наименование ЖК '''
if 'ЖК' in df.columns:
    df = df.drop('ЖК', axis=1)
    
    
''' Проверка на пропущенные значения '''
print('Пропущенные значения по столбцам:')
print(df.isnull().sum())


''' # Проверка на бесконечно малые значения и их обнуление '''
for col in df.select_dtypes(include=['float64', 'int64']).columns:
    small_values = np.abs(df[col]) < 1e-10
    if small_values.any():
        print(f"- Обнаружены бесконечно малые значения в столбце '{col}': {small_values.sum()} значений")
        df.loc[small_values, col] = 0
        print(f"  Бесконечно малые значения в столбце '{col}' обнулены")
print()        


''' Проверка на слишком большие значения (больше 5 млн) '''
for col in df.select_dtypes(include=['float64', 'int64']).columns:
    large_values = df[col] > 5_000_000
    if large_values.any():
        print(f"- Обнаружены слишком большие значения (>5 млн) в столбце '{col}':")
        print(df.loc[large_values, col])
        
    # Если это целевая переменная price_per_sqm, просто выводим информацию
    if col == 'price_per_sqm':
        print(f"- Количество выбросов в целевой переменной 'price_per_sqm': {large_values.sum()}")
        # Удаляем строки с очень большими ценами
        df = df[~large_values]
        print('  Строки с очень большими ценами удалены')
print()

            
''' Проверка на NaN, бесконечности и очень большие значения в price_per_sqm '''
if 'price_per_sqm' in df.columns:
    
    # Проверка на NaN
    nan_prices = df['price_per_sqm'].isna()
    if nan_prices.any():
        print(f"- Обнаружены NaN значения в 'price_per_sqm': {nan_prices.sum()} значений")
        # Удаляем строки с NaN ценами
        df = df[~nan_prices]
        print('  Строки с NaN ценами удалены\n')
 
        
    # Проверка на бесконечности
    inf_prices = ~np.isfinite(df['price_per_sqm'])
    if inf_prices.any():
        print(f"- Обнаружены бесконечные значения в 'price_per_sqm': {inf_prices.sum()} значений")
        # Удаляем строки с бесконечными ценами
        df = df[~inf_prices]
        print('  Строки с бесконечными ценами удалены\n') 
        
    # Проверка на отрицательные значения
    negative_prices = df['price_per_sqm'] <= 0
    if negative_prices.any():
        print(f"- Обнаружены отрицательные или нулевые значения в 'price_per_sqm': {negative_prices.sum()} значений")
        # Удаляем строки с отрицательными ценами
        df = df[~negative_prices]
        print('  Строки с отрицательными или нулевыми ценами удалены\n')
    
    print()


''' Автоматическое определение числовых столбцов и заполнение пропусков медианой '''
numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
for col in numeric_columns:
    missing_values = df[col].isna()
    if missing_values.any():
        print(f"Заполнение пропусков в столбце '{col}': {missing_values.sum()} значений")
        df[col] = df[col].fillna(df[col].median())


''' Автоматическое определение категориальных столбцов и заполнение пропусков модой '''
categorical_columns = df.select_dtypes(include=['object', 'category']).columns.tolist()
for col in categorical_columns:
    missing_values = df[col].isna()
    if missing_values.any():
        # Находим наиболее часто встречающееся значение (моду)
        most_frequent = df[col].mode()[0]
        print(f"Заполнение пропусков в столбце '{col}': {missing_values.sum()} значений значением '{most_frequent}'")
        df[col] = df[col].fillna(most_frequent)


''' Заполнение пропусков в столбцах застройщиков '''
developer_columns = [col for col in df.columns if col.startswith('developer_')]
for col in developer_columns:
    if col in df.columns:
        missing_values = df[col].isna()
        if missing_values.any():
            print(f"Заполнение пропусков в столбце '{col}': {missing_values.sum()} значений")
            df[col] = df[col].fillna(0)


''' Заполнение пропусков в столбцах банков '''
bank_columns = [col for col in df.columns if col.startswith('bank_')]
for col in bank_columns:
    if col in df.columns:
        missing_values = df[col].isna()
        if missing_values.any():
            print(f"Заполнение пропусков в столбце '{col}': {missing_values.sum()} значений")
            df[col] = df[col].fillna(0)
    
            
''' Проверка на оставшиеся пропуски '''
remaining_nulls = df.isnull().sum()
if remaining_nulls.sum() > 0:
    print("\nОставшиеся пропущенные значения после обработки:")
    print(remaining_nulls[remaining_nulls > 0])
    
    # Заполняем оставшиеся пропуски медианой для числовых и наиболее частым значением для категориальных
    for col in df.columns:
        if df[col].isnull().sum() > 0:
            if df[col].dtype in ['float64', 'int64']:
                df[col] = df[col].fillna(df[col].median())
            
            else:
                df[col] = df[col].fillna(df[col].mode()[0])
      
                
''' Базовая статистика по целевой переменной '''
if 'price_per_sqm' in df.columns:
    print("\nСтатистика по целевой переменной 'price_per_sqm':")
    print(df['price_per_sqm'].describe())

### 2. Более глубл

In [ ]:
import time
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression

In [ ]:
# Проверяем доступность CUDA
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используется устройство: {device}")


# Функция для расчета расстояния по формуле гаверсинусов с использованием PyTorch
'''
Расчет расстояний между точками с использованием PyTorch.
[] Args:
    - lat1, lon1: координаты первой точки (тензоры)
    - lat2, lon2: координаты второй точки (тензоры)  
[] Returns:
    - тензор с расстояниями в километрах
'''
def haversine_distance_batch(lat1, lon1, lat2, lon2):

    # Радиус Земли в километрах
    R = 6371.0
    
    # Преобразуем градусы в радианы
    lat1_rad = torch.deg2rad(lat1)
    lon1_rad = torch.deg2rad(lon1)
    lat2_rad = torch.deg2rad(lat2)
    lon2_rad = torch.deg2rad(lon2)
    
    # Разница в координатах
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    
    # Формула гаверсинусов
    a = torch.sin(dlat/2)**2 + torch.cos(lat1_rad) * torch.cos(lat2_rad) * torch.sin(dlon/2)**2
    c = 2 * torch.atan2(torch.sqrt(a), torch.sqrt(1-a))
    distance = R * c
    
    return distance


# Функция находит ближайшие аналоги для каждого объекта с максимальным использованием GPU и применяет пространственное взвешивание цен аналогов
'''
[] Args:
    - df: исходный датафрейм
    - num_analogs: количество аналогов для поиска
    - batch_size: размер батча для обработки на GPU
    - sigma: параметр для экспоненциального взвешивания расстояний  
[] Returns:
    - DataFrame: исходный датафрейм с добавленными столбцами для аналогов и пространственными метриками
'''
def find_nearest_analogs_gpu_optimized(df, num_analogs=10, batch_size=1000, sigma=2.0):

    print(f"Поиск ближайших аналогов для {len(df)} объектов с GPU-оптимизацией...")
    start_time = time.time()
    
    # Создаем копию исходного датафрейма
    result_df = df.copy()
    
    # Преобразуем строковые данные в числовые идентификаторы
    encoder = LabelEncoder()
    residential_complex_encoded = encoder.fit_transform(df['residential_complex'])
    
    # Создаем столбцы для аналогов
    for i in range(1, num_analogs + 1):
        result_df[f'analog_{i}_price_per_sqm'] = np.nan
        result_df[f'analog_{i}_distance'] = np.nan
        result_df[f'analog_{i}_lat'] = np.nan
        result_df[f'analog_{i}_lng'] = np.nan
        result_df[f'analog_{i}_weight'] = np.nan
    
    # Переносим все данные на GPU для ускорения
    indices_tensor = torch.tensor(df.index.values, device=device)
    class_tensor = torch.tensor(df['class'].values, device=device)
    finishing_tensor = torch.tensor(df['finishing'].values, device=device)
    region_tensor = torch.tensor(df['region'].values, device=device)
    zone_tensor = torch.tensor(df['zone'].values, device=device)
    renovation_tensor = torch.tensor(df['renovation'].values, device=device)
    premises_type_tensor = torch.tensor(df['premises_type'].values, device=device)
    reg_year_tensor = torch.tensor(df['registration_date_year'].values, device=device)
    rc_tensor = torch.tensor(residential_complex_encoded, device=device)
    lat_tensor = torch.tensor(df['lat'].values, device=device)
    lng_tensor = torch.tensor(df['lng'].values, device=device)
    price_tensor = torch.tensor(df['price_per_sqm'].values, device=device)
    
    # Обрабатываем датафрейм по частям для экономии памяти
    for start_idx in tqdm(range(0, len(df), batch_size), desc="Обработка батчей"):
        end_idx = min(start_idx + batch_size, len(df))
        batch_indices = df.index[start_idx:end_idx]
        
        # Для каждого объекта в батче
        for idx in batch_indices:
            i = df.index.get_loc(idx)
            
            # Получаем параметры текущего объекта
            current_class = class_tensor[i]
            current_finishing = finishing_tensor[i]
            current_region = region_tensor[i]
            current_zone = zone_tensor[i]
            current_renovation = renovation_tensor[i]
            current_premises_type = premises_type_tensor[i]
            current_reg_year = reg_year_tensor[i]
            current_rc = rc_tensor[i]
            current_lat = lat_tensor[i]
            current_lng = lng_tensor[i]
            current_idx = indices_tensor[i]
            
            # Фильтруем потенциальные аналоги по критериям - все операции на GPU
            mask = (
                    (class_tensor == current_class) &
                    (finishing_tensor == current_finishing) &
                    (region_tensor == current_region) &
                    (zone_tensor == current_zone) &
                    (renovation_tensor == current_renovation) &
                    (premises_type_tensor == current_premises_type) &
                    (reg_year_tensor <= current_reg_year) &
                    (reg_year_tensor >= current_reg_year - 1) &
                    (rc_tensor != current_rc) & 
                    (indices_tensor != current_idx)
                    )
            
            # Если аналогов недостаточно, ослабляем критерии
            if torch.sum(mask) < num_analogs:
                mask = (
                        (class_tensor == current_class) &
                        (region_tensor == current_region) &
                        (premises_type_tensor == current_premises_type) &
                        (reg_year_tensor <= current_reg_year) &
                        (reg_year_tensor >= current_reg_year - 2) &
                        (rc_tensor != current_rc) & 
                        (indices_tensor != current_idx)
                        )
            
            # Если все еще недостаточно, еще больше ослабляем критерии
            if torch.sum(mask) < num_analogs:
                mask = (
                        (class_tensor == current_class) &
                        (premises_type_tensor == current_premises_type) &
                        (rc_tensor != current_rc) & 
                        (indices_tensor != current_idx)
                        )
            
            # Если нашли потенциальные аналоги
            mask_sum = torch.sum(mask)
            if mask_sum > 0:
                
                # Получаем координаты, цены и индексы аналогов - все на GPU
                analog_lats = lat_tensor[mask]
                analog_lngs = lng_tensor[mask]
                analog_prices = price_tensor[mask]
                analog_indices = indices_tensor[mask]
                
                # Рассчитываем расстояния до всех потенциальных аналогов
                current_lat_expanded = current_lat.expand_as(analog_lats)
                current_lng_expanded = current_lng.expand_as(analog_lngs)
                distances = haversine_distance_batch(current_lat_expanded, current_lng_expanded, analog_lats, analog_lngs)
                
                # Если аналогов больше, чем нужно, выбираем ближайшие
                if mask_sum > num_analogs:
                    # Используем topk для выбора ближайших аналогов - операция на GPU
                    _, indices = torch.topk(distances, num_analogs, largest=False)
                    selected_distances = torch.gather(distances, 0, indices)
                    selected_prices = torch.gather(analog_prices, 0, indices)
                    selected_lats = torch.gather(analog_lats, 0, indices)
                    selected_lngs = torch.gather(analog_lngs, 0, indices)
                    selected_indices = torch.gather(analog_indices, 0, indices)
                else:
                    selected_distances = distances
                    selected_prices = analog_prices
                    selected_lats = analog_lats
                    selected_lngs = analog_lngs
                    selected_indices = analog_indices
                
                # Рассчитываем экспоненциальные веса на основе расстояния - на GPU
                weights = torch.exp(-selected_distances / sigma)
                weights = weights / torch.sum(weights) 
                
                # Переносим результаты на CPU
                distances_np = selected_distances.cpu().numpy()
                prices_np = selected_prices.cpu().numpy()
                lats_np = selected_lats.cpu().numpy()
                lngs_np = selected_lngs.cpu().numpy()
                weights_np = weights.cpu().numpy()
                
                # Заполняем столбцы для аналогов
                for j in range(min(num_analogs, len(distances_np))):
                    result_df.at[idx, f'analog_{j+1}_price_per_sqm'] = prices_np[j]
                    result_df.at[idx, f'analog_{j+1}_distance'] = distances_np[j]
                    result_df.at[idx, f'analog_{j+1}_lat'] = lats_np[j]
                    result_df.at[idx, f'analog_{j+1}_lng'] = lngs_np[j]
                    result_df.at[idx, f'analog_{j+1}_weight'] = weights_np[j]
    
    # Вычисляем агрегированные метрики
    print("Расчет агрегированных метрик...")
    
    # Подготавливаем названия столбцов
    analog_price_cols = [f'analog_{i}_price_per_sqm' for i in range(1, num_analogs + 1)]
    analog_distance_cols = [f'analog_{i}_distance' for i in range(1, num_analogs + 1)]
    analog_weight_cols = [f'analog_{i}_weight' for i in range(1, num_analogs + 1)]
    analog_lat_cols = [f'analog_{i}_lat' for i in range(1, num_analogs + 1)]
    analog_lng_cols = [f'analog_{i}_lng' for i in range(1, num_analogs + 1)]
    
    # Обрабатываем строки с отсутствующими аналогами
    for col in analog_price_cols + analog_distance_cols + analog_weight_cols + analog_lat_cols + analog_lng_cols:
        result_df[col] = pd.to_numeric(result_df[col], errors='coerce')
    
    # 1. Базовые агрегированные метрики
    result_df['analogs_mean_price'] = result_df[analog_price_cols].mean(axis=1)
    result_df['analogs_median_price'] = result_df[analog_price_cols].median(axis=1)
    result_df['analogs_min_price'] = result_df[analog_price_cols].min(axis=1)
    result_df['analogs_max_price'] = result_df[analog_price_cols].max(axis=1)
    result_df['analogs_std_price'] = result_df[analog_price_cols].std(axis=1)
    result_df['analogs_mean_distance'] = result_df[analog_distance_cols].mean(axis=1)
    
    # 2. Взвешенные метрики на основе расстояния
    # Используем векторизованные операции pandas
    result_df['analogs_weighted_mean_price'] = (
        result_df[analog_price_cols].values * result_df[analog_weight_cols].values
    ).sum(axis=1)
    
    # 3. Пространственные характеристики распределения аналогов
    result_df['analogs_lat_std'] = result_df[analog_lat_cols].std(axis=1)
    result_df['analogs_lng_std'] = result_df[analog_lng_cols].std(axis=1)
    
    # Направление распределения аналогов (север-юг или восток-запад)
    result_df['analogs_direction_ratio'] = result_df['analogs_lat_std'] / (result_df['analogs_lng_std'] + 0.001)
    
    # 4. Рассчитываем отклонение цены объекта от взвешенной средней аналогов
    result_df['price_deviation_from_weighted_analogs'] = (result_df['price_per_sqm'] - result_df['analogs_weighted_mean_price']) / result_df['analogs_weighted_mean_price']
    
    # 5. Градиент цен - ИСПРАВЛЕННАЯ ВЕРСИЯ с использованием sklearn
    print("Расчет градиентов цен с использованием sklearn...")
    
    # Подготавливаем столбцы для градиентов
    result_df['price_gradient_lat'] = 0.0  # Инициализируем нулями вместо NaN
    result_df['price_gradient_lng'] = 0.0
    result_df['price_gradient_magnitude'] = 0.0
    result_df['price_gradient_direction'] = 0.0
    
    # Обрабатываем датафрейм по частям для экономии памяти
    for start_idx in tqdm(range(0, len(result_df), batch_size), desc="Расчет градиентов по батчам"):
        end_idx = min(start_idx + batch_size, len(result_df))
        batch_indices = result_df.index[start_idx:end_idx]
        
        for idx in batch_indices:
            # Получаем данные текущего объекта и его аналогов
            row = result_df.loc[idx]
            
            # Собираем данные для расчета градиента
            X_data = []
            y_data = []
            
            for i in range(1, num_analogs + 1):
                
                # Проверяем, что все необходимые данные аналога присутствуют
                if (not pd.isna(row[f'analog_{i}_lat']) and 
                    not pd.isna(row[f'analog_{i}_lng']) and 
                    not pd.isna(row[f'analog_{i}_price_per_sqm'])):
                    
                    # Разница в координатах (аналог - объект)
                    lat_diff = row[f'analog_{i}_lat'] - row['lat']
                    lng_diff = row[f'analog_{i}_lng'] - row['lng']
                    
                    # Разница в цене
                    price_diff = row[f'analog_{i}_price_per_sqm'] - row['price_per_sqm']
                    
                    X_data.append([lat_diff, lng_diff])
                    y_data.append(price_diff)
            
            # Если есть достаточно данных для расчета градиента
            if len(X_data) >= 3: 
                try:
                    # Используем sklearn LinearRegression для надежности
                    reg = LinearRegression(fit_intercept=False) 
                    reg.fit(X_data, y_data)
                    
                    # Получаем коэффициенты градиента
                    gradient_lat = reg.coef_[0]
                    gradient_lng = reg.coef_[1]
                    
                    # Рассчитываем магнитуду и направление градиента
                    magnitude = np.sqrt(gradient_lat**2 + gradient_lng**2)
                    direction = np.arctan2(gradient_lat, gradient_lng) * 180 / np.pi
                    
                    # Сохраняем результаты
                    result_df.at[idx, 'price_gradient_lat'] = gradient_lat
                    result_df.at[idx, 'price_gradient_lng'] = gradient_lng
                    result_df.at[idx, 'price_gradient_magnitude'] = magnitude
                    result_df.at[idx, 'price_gradient_direction'] = direction
                    
                except Exception as e:
                    # В случае ошибки просто пропускаем (оставляем нули)
                    print(f"Ошибка при расчете градиента для {idx}: {e}")
    
    # 6. Добавляем различные комбинации признаков с векторизацией
    print("Расчет комбинированных признаков...")
    
    # Взаимодействие между градиентом цены и расстоянием
    result_df['gradient_distance_interaction'] = result_df['price_gradient_magnitude'] * result_df['analogs_mean_distance']
    
    # Относительное положение объекта в градиенте цен - векторизованная реализация
    result_df['relative_position_in_gradient'] = 0
    
    # Используем векторизацию для ускорения
    for i in range(1, num_analogs + 1):
        # Проекция вектора от аналога к объекту на вектор градиента
        lat_diff = result_df['lat'] - result_df[f'analog_{i}_lat']
        lng_diff = result_df['lng'] - result_df[f'analog_{i}_lng']
        
        # Нормализованный вектор градиента
        grad_magnitude = result_df['price_gradient_magnitude']
        grad_lat = result_df['price_gradient_lat'] / (grad_magnitude + 0.001)
        grad_lng = result_df['price_gradient_lng'] / (grad_magnitude + 0.001)
        
        # Скалярное произведение
        projection = lat_diff * grad_lat + lng_diff * grad_lng
        result_df['relative_position_in_gradient'] += projection * result_df[f'analog_{i}_weight']
    
    # Заполняем NaN в агрегированных метриках
    agg_cols = [
                'analogs_mean_price', 'analogs_median_price', 'analogs_min_price', 
                'analogs_max_price', 'analogs_std_price', 'analogs_mean_distance',
                'analogs_weighted_mean_price', 'analogs_lat_std', 'analogs_lng_std',
                'analogs_direction_ratio', 'price_gradient_lat', 'price_gradient_lng',
                'price_gradient_magnitude', 'price_gradient_direction',
                'price_deviation_from_weighted_analogs', 'gradient_distance_interaction',
                'relative_position_in_gradient'
                ]
    
    # Заменяем NaN медианными значениями (более робастно, чем среднее)                       
    for col in agg_cols:
        median_value = result_df[col].median()
        if pd.isna(median_value):
            median_value = 0
        result_df[col] = result_df[col].fillna(median_value)
    
    end_time = time.time()
    print(f"Обработка завершена за {end_time - start_time:.2f} секунд.")
    return result_df